In [1]:
import sys
import os
import yaml
from dataclasses import asdict

sys.path.append("./CLAP")
current_dir = os.getcwd()

In [2]:
mo833_nome_do_usuario = "professor" # Modifique esta variável com o seu usuário de nuvem mo833-...

In [3]:
from app.cli.modules.node import get_config_db, get_node_manager
from app.cli.modules.role import get_role_manager
from app.cli.modules.cluster import get_cluster_config_db, get_cluster_manager
from clap.utils import float_time_to_string

In [4]:
instance_config_db = get_config_db()
cluster_config_db = get_cluster_config_db()
node_manager = get_node_manager()
role_manager = get_role_manager()
cluster_manager = get_cluster_manager()
private_path = node_manager.private_path

Discarding role '.ipynb_checkpoints'. IsADirectoryError: [Errno 21] Is a directory: '/home/staff/edson/2022-mo833/ativ-9/mo833-atividade9/clap_config/roles/actions.d/.ipynb_checkpoints'
Discarding role '.ipynb_checkpoints'. IsADirectoryError: [Errno 21] Is a directory: '/home/staff/edson/2022-mo833/ativ-9/mo833-atividade9/clap_config/roles/actions.d/.ipynb_checkpoints'


In [5]:
t2_medium_type = instance_config_db.instance_descriptors["type-t2.medium"]
asdict(t2_medium_type)

{'provider': {'provider_config_id': 'aws-config-us-east-1',
  'region': 'us-east-1',
  'access_keyfile': 'ec2_access_key.pub',
  'secret_access_keyfile': 'ec2_access_key.pem',
  'vpc': None,
  'url': None,
  'provider': 'aws'},
 'login': {'login_config_id': 'login-ubuntu',
  'user': 'ubuntu',
  'keypair_name': 'mo833-teacher',
  'keypair_public_file': 'key.pub',
  'keypair_private_file': 'key.pem',
  'ssh_port': 22,
  'sudo': True,
  'sudo_user': 'root'},
 'instance': {'instance_config_id': 'type-t2.medium',
  'provider': 'aws-config-us-east-1',
  'login': 'login-ubuntu',
  'flavor': 't2.medium',
  'image_id': 'ami-0c4f7023847b90238',
  'security_group': 'open-security-group',
  'boot_disk_size': 16,
  'boot_disk_device': None,
  'boot_disk_type': None,
  'boot_disk_iops': None,
  'boot_disk_snapshot': None,
  'placement_group': None,
  'price': None,
  'timeout': None,
  'network_ids': []}}

In [6]:
node_ids = node_manager.start_node(t2_medium_type, count=2)
print(f"Nodes {', '.join(node_ids)} were started!")

[WARNING]: provided hosts list is empty, only localhost is available. Note that
the implicit localhost does not match 'all'
[DEPRECATION WARNING]: amazon.aws.ec2 has been deprecated. The ec2 module is 
based upon a deprecated version of the AWS SDKs and is deprecated in favor of 
the ec2_instance module. Please update your tasks. This feature will be removed
 from amazon.aws in version 4.0.0. Deprecation warnings can be disabled by 
setting deprecation_warnings=False in ansible.cfg.

PLAY [localhost] ***************************************************************

TASK [Starting 2 type-t2.medium instances (timeout 600 seconds)] ***************
[DEPRECATION WARNING]: The 'ec2' module has been deprecated and replaced by the
 'ec2_instance' module'. This feature will be removed from amazon.aws in 
version 4.0.0. Deprecation warnings can be disabled by setting 
deprecation_warnings=False in ansible.cfg.
changed: [localhost]

PLAY RECAP ******************************************************

In [7]:
alive_nodes = node_manager.is_alive(node_ids)
for node_id, alive_flag in alive_nodes.items():
    alive_str = 'alive' if alive_flag == True else 'not alive'
    print(f"{node_id} --> {alive_str}.")

Error executing command in node 5641c651: [Errno None] Unable to connect to port 22 on 52.201.233.44
Error executing command in node 62cc58aa: [Errno None] Unable to connect to port 22 on 54.145.128.31
Error executing command in 5641c651: [Errno None] Unable to connect to port 22 on 52.201.233.44.
Error executing command in 62cc58aa: [Errno None] Unable to connect to port 22 on 54.145.128.31.


5641c651ff1b4bd7b91ab218aeb1146e --> alive.
62cc58aaa4774dbc906f3a09fff63a91 --> alive.


In [8]:
# Variaveis espeficica por nó
node_vars={
    node_ids[0]: {
        "node_rank": 0
    }, 
    node_ids[1]: {
        "node_rank": 1
    }
}

# Variaveis passadas para a ação setup
extra_args = {
    "efs_host": "172.31.15.9",
    "efs_mount_point": "/home/ubuntu/efs/",
    "user_dir": f"/home/ubuntu/efs/{mo833_nome_do_usuario}/"
}


nodes_added_to_role = role_manager.add_role(
    role_name="dcgan", 
    hosts_node_map=node_ids,
    node_vars=node_vars,
    extra_args=extra_args,
)

print(f"O papel dcgan foram adicionados aos nós {', '.join(nodes_added_to_role)}")


PLAY [Acao de setup] ***********************************************************

TASK [Gathering Facts] *********************************************************
ok: [0414f2c0cf0648858d452775b65d6ea8]
ok: [df88a757a6ac438f91170caaec65a360]

TASK [Task 1: Lista as variáveis recebidas] ************************************
ok: [df88a757a6ac438f91170caaec65a360] => {
    "msg": "Variaveis: efs_host: 172.31.15.9; efs_mount_point: /home/ubuntu/efs/; user_dir: /home/ubuntu/efs/professor/"
}
ok: [0414f2c0cf0648858d452775b65d6ea8] => {
    "msg": "Variaveis: efs_host: 172.31.15.9; efs_mount_point: /home/ubuntu/efs/; user_dir: /home/ubuntu/efs/professor/"
}

TASK [Task 2: Cria o grupo docker] *********************************************
ok: [df88a757a6ac438f91170caaec65a360]
ok: [0414f2c0cf0648858d452775b65d6ea8]

TASK [Task 3: Adicionar usuario ubuntu ao grupo docker] ************************
ok: [0414f2c0cf0648858d452775b65d6ea8]
ok: [df88a757a6ac438f91170caaec65a360]

TASK [Task 4: Reinici

In [10]:
# Variaveis espeficica por nó
node_vars={
    node_ids[0]: {
        "node_rank": 0
    }, 
    node_ids[1]: {
        "node_rank": 1
    }
}

# Variaveis passadas para a ação setup
extra_args = {
    "efs_mount_point": "/home/ubuntu/efs/",
    "user_dir": f"/home/ubuntu/efs/{mo833_nome_do_usuario}/",
    "cifar_data_dir": "/home/ubuntu/efs/data/cifar10/",
    "master_hostname": node_ids[0],
}

res = role_manager.perform_action(
    role_name="dcgan", 
    action_name="run", 
    hosts_node_map=node_ids, 
    node_vars=node_vars,
    extra_args=extra_args
)

print(f"Playbook executou bem? {res.ok}")


PLAY [Run Play] ****************************************************************

TASK [Task 1: Lista as variáveis recebidas] ************************************
ok: [df88a757a6ac438f91170caaec65a360] => {
    "msg": "Variaveis: efs_mount_point: /home/ubuntu/efs/; user_dir: /home/ubuntu/efs/professor/; cifar_data_dir: /home/ubuntu/efs/data/cifar10/; master_hostname: df88a757a6ac438f91170caaec65a360"
}
ok: [0414f2c0cf0648858d452775b65d6ea8] => {
    "msg": "Variaveis: efs_mount_point: /home/ubuntu/efs/; user_dir: /home/ubuntu/efs/professor/; cifar_data_dir: /home/ubuntu/efs/data/cifar10/; master_hostname: df88a757a6ac438f91170caaec65a360"
}

PLAY RECAP *********************************************************************
0414f2c0cf0648858d452775b65d6ea8 : ok=1    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
df88a757a6ac438f91170caaec65a360 : ok=1    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
Playbook executou be

In [11]:
# Variaveis espeficica por nó
node_vars={
    node_ids[0]: {
        "node_rank": 0
    }, 
    node_ids[1]: {
        "node_rank": 1
    }
}

# Variaveis passadas para a ação setup
extra_args = {
    "user_dir": f"/home/ubuntu/efs/{mo833_nome_do_usuario}/",
    "results_dir": f"{current_dir}/resultados"
}


res = role_manager.perform_action(
    role_name="dcgan", 
    action_name="results", 
    hosts_node_map=node_ids, 
    node_vars=node_vars,
    extra_args=extra_args,
    quiet=False
)

print(f"Playbook executou bem? {res.ok}")


PLAY [Acao de resultados] ******************************************************

TASK [Gathering Facts] *********************************************************
ok: [df88a757a6ac438f91170caaec65a360]
ok: [0414f2c0cf0648858d452775b65d6ea8]

TASK [Task 1: Lista as variáveis recebidas] ************************************
ok: [df88a757a6ac438f91170caaec65a360] => {
    "msg": "Variaveis: results_dir: /home/staff/edson/2022-mo833/ativ-9/mo833-atividade9/resultados; user_dir: /home/ubuntu/efs/professor/"
}
ok: [0414f2c0cf0648858d452775b65d6ea8] => {
    "msg": "Variaveis: results_dir: /home/staff/edson/2022-mo833/ativ-9/mo833-atividade9/resultados; user_dir: /home/ubuntu/efs/professor/"
}

PLAY RECAP *********************************************************************
0414f2c0cf0648858d452775b65d6ea8 : ok=2    changed=0    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
df88a757a6ac438f91170caaec65a360 : ok=2    changed=0    unreachable=0    failed=0    skipped=0    

In [12]:
stopped_nodes = node_manager.stop_nodes(node_ids)
print(f"Stopped nodes: {', '.join(stopped_nodes)}")

[WARNING]: provided hosts list is empty, only localhost is available. Note that
the implicit localhost does not match 'all'
[DEPRECATION WARNING]: amazon.aws.ec2 has been deprecated. The ec2 module is 
based upon a deprecated version of the AWS SDKs and is deprecated in favor of 
the ec2_instance module. Please update your tasks. This feature will be removed
 from amazon.aws in version 4.0.0. Deprecation warnings can be disabled by 
setting deprecation_warnings=False in ansible.cfg.

PLAY [localhost] ***************************************************************

TASK [Stopping nodes KathyMaisenbacher, MaryMiller] ****************************
deprecation_warnings=False in ansible.cfg.
changed: [localhost]

PLAY RECAP *********************************************************************
localhost                  : ok=1    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
Stopped nodes: df88a757a6ac438f91170caaec65a360, 0414f2c0cf0648858d452775b65d6ea8
